In [2]:
# Data prep
import os, shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from random import random
import pandas as pd
import re
keras.mixed_precision.set_global_policy('mixed_float16')

# data loading config
batch_size = 64
img_height = 256
img_width = 256
dataPath = 'oxford_flower_102'
labels = 'inferred'
label_mode = 'categorical'  # one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE

train_df = pd.read_csv('image_to_label.csv', names=['picture', 'label'])
test_df = pd.DataFrame(columns=['picture', 'label'])
with open('102_flower_labels.txt') as f:
    regex = re.compile('[^a-zA-Z\s-]')
    classes=[]
    for line in f:
        classes += [regex.sub('', line.replace('\'', '').replace(' ', '').strip())]  # remove bullshit


def split_train_test(dataPath):
    for root, dirs, files in os.walk(dataPath):
        for name in files:
            randomNum = random()
            row = train_df.iloc[train_df.index[train_df['picture'] == name]]
            if randomNum <= test_split:
                os.makedirs('test\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'test\\'+classes[row.label.tolist()[0]-1]+'\\')
            else:
                os.makedirs('train\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'train\\'+classes[row.label.tolist()[0]-1]+'\\')

# split_train_test(dataPath)  # Only need to run this once

print('Training data')
train = keras.preprocessing.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, subset="training", seed=seed, validation_split=test_split,
image_size=(img_height, img_width), batch_size=batch_size)
train_class_names = train.class_names

val = keras.preprocessing.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, subset="validation", seed=seed, validation_split=test_split,
image_size=(img_height, img_width), batch_size=batch_size)

print('\nTesting data')
test = keras.preprocessing.image_dataset_from_directory('test', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed,
image_size=(img_height, img_width), batch_size=batch_size)
test_class_names = test.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

print()
print('train classes:', train_class_names)
print('test classes:', test_class_names)

Training data
Found 6520 files belonging to 102 classes.
Using 5216 files for training.
Found 6520 files belonging to 102 classes.
Using 1304 files for validation.

Testing data
Found 1669 files belonging to 102 classes.

train classes: ['alpineseaholly', 'anthurium', 'artichoke', 'azalea', 'ballmoss', 'balloonflower', 'barbetondaisy', 'beardediris', 'beebalm', 'birdofparadise', 'bishopofllandaff', 'black-eyedsusan', 'blackberrylily', 'blanketflower', 'bolerodeepblue', 'bougainvillea', 'bromelia', 'buttercup', 'californianpoppy', 'camellia', 'cannalily', 'canterburybells', 'capeflower', 'carnation', 'cautleyaspicata', 'clematis', 'coltsfoot', 'columbine', 'commondandelion', 'cornpoppy', 'cyclamen', 'daffodil', 'desert-rose', 'englishmarigold', 'firelily', 'foxglove', 'frangipani', 'fritillary', 'gardenphlox', 'gaura', 'gazania', 'geranium', 'giantwhitearumlily', 'globe-flower', 'globethistle', 'grapehyacinth', 'greatmasterwort', 'hard-leavedpocketorchid', 'hibiscus', 'hippeastrum', 'ja